In [1]:
import pickle
import pandas as pd

enron_df = pd.read_pickle('enron_students.pkl')

# # Read dictionary pkl file
# with open('email_to_departments.pkl', 'rb') as fp:
#     email_to_departments = pickle.load(fp)
#     print('email_to_departments dictionary loaded from pkl file')

In [2]:
# data cleaning
enron_df['To'] = enron_df['To'].fillna('')
enron_df['From'] = enron_df['From'].fillna('')
enron_df['X-From'] = enron_df['X-From'].fillna('')
enron_df['X-To'] = enron_df['X-To'].fillna('')
enron_df['X-cc'] = enron_df['X-cc'].fillna('')
enron_df['X-bcc'] = enron_df['X-bcc'].fillna('')


In [3]:
import re
import pandas as pd

def extract_emails(text):
    if pd.isna(text):
        return []
    # Regex to match email addresses
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    return re.findall(email_pattern, text)

def extract_single_email_from_text(text):
    emails = extract_emails(text)
    if (len(emails) > 1):
        print(f"Multiple emails found in: {text}")
    elif len(emails) == 1:
        return emails[0]
    print(f"Could not extract email from: {text}")
    return None

def extract_sent_to_emails(text):
    emails = extract_emails(text)
    if len(emails) > 0:
        return emails
    return []

def is_between_ect_and_ees(email_to_depart_dict, row, is_including_cc_bcc=False, additional_check=True):
    sender_email = extract_single_email_from_text(row['From'])
    if sender_email in email_to_depart_dict:
        sender_department = email_to_depart_dict[sender_email]
        if sender_department != 'ECT' and sender_department != 'EES':
            return False
        else:
            if is_including_cc_bcc:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
            else:
                reciever_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To']) + extract_sent_to_emails(row['X-cc']) + extract_sent_to_emails(row['X-bcc'])
            
            for email in reciever_emails:
                if email in email_to_depart_dict:
                    reciever_department = email_to_depart_dict[email]

                    if sender_department == 'ECT' and reciever_department == 'EES':
                        return True
                    elif sender_department == 'EES' and reciever_department == 'ECT':
                        return True
                    
            if additional_check:
                if 'ECT' in row['X-To']:
                    if 'EES' in row['X-From']:
                        return True
                if 'EES' in row['X-To']:
                    if 'ECT' in row['X-From']:
                        return True
      
            return False
    
    else:
        if additional_check:
            if 'ECT' in row['X-To']:
                if 'EES' in row['X-From']:
                    return True
            if 'EES' in row['X-To']:
                if 'ECT' in row['X-From']:
                    return True
        return False
    


    

In [4]:

def build_dict():
    email_to_departments = {}
    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com': 'EES'
            })
    
    email_to_departments.update({'eric.bass@enron.com': 'ECT'})

    multiple_count = 0
    # TODO: implement check for multiple emails in X-From and From. should be only one email
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        to_emails = set([mail.lower() for mail in (extract_emails(row['To']) + extract_emails(row['X-To']))])
        if len(from_emails) == 0:
            continue
        if len(from_emails) > 1:
            from_emails = [mail for mail in from_emails if not mail.startswith('imceanotes')]
            if len(from_emails) > 1 or len(from_emails) == 0:
                multiple_count += 1

        for email in from_emails:
            email = email.lower()
            # TODO: validate company email starts with enron.com
            if "enron.com" not in email and email not in email_to_departments:
                email_to_departments.update({email: 'NA'})
            else:
                if email in email_to_departments and email_to_departments[email] != 'Other':
                    continue
                elif 'EES' in row['X-From']:
                    email_to_departments.update({email: 'EES'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'ECT'})
                elif 'ECT' in row['X-From']:
                    email_to_departments.update({email: 'ECT'})
                    if len(to_emails) == 1:
                        email_to_departments.update({list(to_emails)[0]: 'EES'})
                else:
                    email_to_departments.update({email: 'Other'})

        # new addition - need to check
        if len(to_emails) == 1:
            if 'ECT' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'ECT'})
            elif 'EES' in row['X-To']:
                email_to_departments.update({list(to_emails)[0]: 'EES'})

    email_to_departments.update({
            'smith.day@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stuart.zisman@enron.com' : 'ECT'
            ,'melissa.murphy@enron.com' : 'ECT'
            ,'thresa.allen@enron.com' : 'ECT'
            ,'leslie.reeves@enron.com' : 'ECT'
            ,'stacey.white@enron.com' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'brad.nebergall@enron.com': 'ECT'
            ,'colleen.koenig@enron.com': 'EES'
            ,'john.kinser@enron.com': 'ECT'
            ,'jeff.merola@enron.com	': 'EES'
            ,'scott.mills@enron.com': 'ECT'
            ,'marilyn.colbert@enron.com': 'ECT'
            ,'molly.harris@enron.com': 'ECT'
            ,'joseph.wagner@enron.com': 'ECT'
            ,'stuart.zisman@enron.com,' : 'ECT'
            ,'janet.wallis@enron.com' : 'ECT'
            ,'daren.farmer@enron.com' : 'ECT'
            ,'stephanie.gardner@enron.com' : 'EES'

            })
    
    return email_to_departments


email_to_department_dict = build_dict()

In [5]:
len(email_to_department_dict)

20638

In [6]:
enron_df['is_between_ect_and_ees'] = enron_df.apply(lambda row: is_between_ect_and_ees(email_to_department_dict, row), axis=1)

Could not extract email from: u@d.h
Could not extract email from: pep <performance.>


In [7]:
# remove all spaces 
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.replace(' ', ''))

# apply split ',' on violated rules
enron_df['violated_rules'] = enron_df['violated_rules'].apply(lambda x: x.split(',')) 

In [8]:
def update_dict(enron_df_not_catched_1_1, email_to_depart):
    # iterate over the rows of the dataframe enron_df_not_catched_1_1
    updated = 0
    for index, row in enron_df_not_catched_1_1.iterrows():
        x_emails = extract_emails(row['X-From'])
        emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        sent_to_emails = extract_sent_to_emails(row['To']) + extract_sent_to_emails(row['X-To'])
        if len(emails) == 0:
            continue
        if len(emails) > 1:
            emails = [mail for mail in emails if not mail.startswith('imceanotes')]
            if len(emails) > 1 or len(emails) == 0:
                # print(f"From: {row['From']} X-From: {row['X-From']} has multiple emails: {emails}")
                continue
        for email in emails:
            email = email.lower()
            # if from email is in dictionary, get the department
            if email in email_to_depart:
                depart = email_to_depart[email]
                if depart == 'ECT':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'EES'
                        updated += 1
                elif depart == 'EES':
                    if len(sent_to_emails) == 1:
                        email_to_depart[sent_to_emails[0]] = 'ECT'
                        updated += 1

    print(f"Updated {updated} emails")
    


In [9]:
enron_df_not_catched_1_1 = enron_df[enron_df['is_between_ect_and_ees'] == False & enron_df['violated_rules'].apply(lambda x: '1.1' in x)]

update_dict(enron_df_not_catched_1_1, email_to_department_dict)

Updated 198084 emails


* HERE WE START USER TO LOCATION *


In [10]:
def get_email_to_location_dict(df):
    email_to_location = {}
    for index, row in enron_df.iterrows():
        x_emails = extract_emails(row['X-From'])
        from_emails = set([mail.lower() for mail in (extract_emails(row['From']) + x_emails)])
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})

    # iterate over the dataframe to get 
    violated_rule_1_2 = df[df['violated_rules'].apply(lambda x: '1.2' in x)]

    for index, row in violated_rule_1_2.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        if '/OU=EU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'EU'})
        elif '/HOU/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        elif '/OU=NA/' in row['X-From']:
            for email in from_emails:
                email_to_location.update({email: 'NA'})
        
        sent_to_email = extract_emails(row['X-To'])
        if len(sent_to_email) == 1:
            if from_emails[0] not in email_to_location:
                continue
            if email_to_location[from_emails[0]] == 'EU':
                email_to_location.update({sent_to_email[0]: 'NA'})
            elif email_to_location[from_emails[0]] == 'NA':
                email_to_location.update({sent_to_email[0]: 'EU'})
            
    return email_to_location


email_to_location = get_email_to_location_dict(enron_df)

In [11]:
def is_between_EU_and_NA(row, email_to_location):
    from_email = extract_emails(row['From'])
    from_email = extract_emails(row['X-From']) + from_email
    from_email = list(set([mail.lower() for mail in from_email]))
    to_email = extract_emails(row['To'])
    to_email = extract_emails(row['X-To']) + to_email
    to_email = list(set([mail.lower() for mail in to_email]))
    if len(from_email) == 0 or len(to_email) == 0:
        return False
    if from_email[0] in email_to_location:
        if email_to_location[from_email[0]] == 'EU':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'NA':
                    return True
        if email_to_location[from_email[0]] == 'NA':
            for email in to_email:
                if email in email_to_location and email_to_location[email] == 'EU':
                    return True
    
    return False

In [12]:
enron_df['is_EU_To_NA'] = enron_df.apply(lambda row: is_between_EU_and_NA(row, email_to_location), axis=1)

In [3]:
not_working = enron_df[(enron_df['is_EU_To_NA'] == False) & (enron_df['violated_rules'].apply(lambda x: '1.2' in x))]
not_working.head(10)

NameError: name 'enron_df' is not defined

In [43]:
print(len(email_to_location))
# enron_df.head(10)
pd.set_option('display.max_colwidth', 50)

3985


In [179]:
def get_c_suit_dict():
    emails = set({
        'kenneth.lay@enron.com',
        'ken.skilling@enron.com',
        'rbowers@nyiso.com',
        'michael.brown@enron.com',
        'john.sherriff@enron.com',
        'jeffrey.sherrick@enron.com',
        'a..shankman@enron.com',
        'ken.rice@enron.com',
        'greg.piper@enron.com',
        'mark.metts@enron.com',
        'coo.jeff@enron.com',
        'rebecca.mcdonald@enron.com',
        'danny.mccarty@enron.com',
        'dan.leff@enron.com',
        'john.lavorato@enron.com',
        'mark.koenig@enron.com',
        'louise.kitchen@enron.com',
        'stanley.horton@enron.com',
        '40enron@enron.com', # for some reason is tagged for stanley horton
        'ben.glisan@enron.com',
        'mark.frevert@enron.com',
        'andrew.fastow@enron.com',
        'jr..legal@enron.com',
        'derrick@enron.com',
        'david.delainey@enron.com',
        'richard.causey@enron.com',
        'michael.brown@enron.com',
        'raymond.bowen@enron.com',
    })

    violated_rule_1_3 = enron_df[enron_df['violated_rules'].apply(lambda x: '2.2' in x)]

    for index, row in violated_rule_1_3.iterrows():
        from_emails = extract_emails(row['X-From'])
        from_emails = list(set([mail.lower() for mail in (extract_emails(row['From']) + from_emails)]))
        to_email = extract_emails(row['To'])
        to_email = extract_emails(row['X-To']) + to_email
        to_email = list(set([mail.lower() for mail in to_email]))

        if len(from_emails) == 0:
            continue
        for email in from_emails:
            emails.add(email)
            # print(f"Email: {email} is in C-Suit")

        # for email in to_email:
        #     emails.add(email)
        #     print(f"Email: {email} is in C-Suit")

    return emails

c_suit_emails = get_c_suit_dict()

In [14]:
def is_in_c_suit(row, c_suit_emails):
    # Extract and clean email addresses from 'From' fields
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    # Check if both sender and recipient are in the c-suite email list
    return any(email in c_suit_emails for email in from_emails) and any(email in c_suit_emails for email in to_email)



In [15]:
enron_df['is_c_suit'] = enron_df.apply(lambda row: is_in_c_suit(row, c_suit_emails), axis=1)

In [16]:
def get_from_to_emails(row):
    from_emails = extract_emails(row['X-From']) + extract_emails(row['From'])
    from_emails = list(set(email.lower() for email in from_emails))

    # Extract and clean email addresses from 'To' fields
    to_email = extract_emails(row['X-To']) + extract_emails(row['To'])
    to_email = list(set(email.lower() for email in to_email))

    return from_emails, to_email

In [17]:
def is_leaving_corporate(row):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False
    
    for email in to_email:
        if 'enron.com' not in email:
            return True

    return False


def is_leaving_ect(row, email_to_depart):
    from_emails, to_email = get_from_to_emails(row)
    if len(from_emails) == 0 or len(to_email) == 0:
        return False

    for email in from_emails:
        if email in email_to_depart and email_to_depart[email] == 'ECT':
            for t_email in to_email:
                if t_email in email_to_depart and email_to_depart[t_email] != 'ECT':
                    return True

    return False

In [18]:
class StaticAnalyzer:
    
    @staticmethod
    def is_violating_rule_1_1(row):
        return is_between_ect_and_ees(email_to_department_dict, row, is_including_cc_bcc=False, additional_check=True)
    
    
    @staticmethod
    def is_violating_rule_1_2(row):
        return is_between_EU_and_NA(row, email_to_location)
    
    
    @staticmethod
    def is_violating_rule_1_3(row):
        return is_leaving_corporate(row)
    
    
    @staticmethod
    def is_violating_rule_2_1(row):
        return is_leaving_ect(row, email_to_department_dict)
    
    
    @staticmethod
    def is_violating_rule_2_2(row):
        return is_in_c_suit(row, c_suit_emails)
    
    @staticmethod
    def is_violating_rule_2_3(row):
        return is_leaving_corporate(row)

In [ ]:
import dataclasses

@dataclasses.dataclass()
class ContentAnalysisResult:
    quids = []
    piis = []

    def __str__(self):
        return f'(Sensitive: {len(self.piis) > 0 or len(self.quids) > 0} | Quids: {self.quids} | PII: {self.piis})'
    
    def is_sensitive(self, rule_id):
        if rule_id == '2.3':
            return len(self.piis) > 0 or len(self.quids) >= 2
        else:
            return len(self.piis) > 0 or len(self.quids) > 0

@dataclasses.dataclass()
class TopicAnalysisResult:
    is_legal: bool = False
    is_business: bool = False
    is_finance: bool = False
    
    def __str__(self):
        return f'(Legal: {self.is_legal} | Business: {self.is_business} | Finance: {self.is_finance})'
    
@dataclasses.dataclass()
class EnforcerResult:
    violated_rules = []
    
    def is_allowed(self)-> bool:
        return len(self.violated_rules) == 0
    
    def __str__(self):
        return f'Violated Rules: {self.violated_rules}'
    

In [ ]:
import spacy 
import re


def find_all_ssn(text):
    return re.findall(r'\d{3}-\d{2}-\d{4}', text)

def find_all_credit_cards(text):
    return re.findall(r'\d{4}-\d{4}-\d{4}-\d{4}', text) 

def find_all_phone_numbers(text):
    return re.findall(r'\(?\d{3}\)?\s*-\s*\d{3}\s*-\s*\d{4}', text) 

def find_sensitive_words(text):
    return re.findall(r'password|attach|confidential', text.lower())

class ContentAnalyzer:
    
    class PII:
        def __init__(self, entity_type, score, text):
            self.entity_type = entity_type
            self.score = score
            self.text = text
            
        def __str__(self):
            return f'(Entity: {self.entity_type} | Score: {self.score} | Text: {self.text})'
    
    MAX_DOC_SIZE_FOR_SPACY = 2000
    QUIDS = ['ORG', 'GPE', 'LOW', 'FAC', 'LOC']
    SENSITIVE = ['MONEY', 'PERCENT', 'NORP', 'PRODUCT', 'EVENT']
    POTENTIALLY_SENSITIVE = ['DATE', 'TIME', 'QUANTITY', 'ORDINAL', 'CARDINAL', 'PERSON']
    
    def __init__(self):
        self.nlp = spacy.load('en_core_web_sm')
        self.nlp.max_length = 1500000
    
    def _get_entities(self, document):
        try:
            doc_len = len(document)
            if doc_len > self.MAX_DOC_SIZE_FOR_SPACY:
                document = document[:self.MAX_DOC_SIZE_FOR_SPACY]
            doc = self.nlp(document)
            return [(ent.text, ent.label_) for ent in doc.ents]
    
        except ValueError as e :
            print(f'Error in document. Error: {e}')
            return []
        
        
    def predict_spacy_verdict(self, row) -> ContentAnalysisResult:
        result = ContentAnalysisResult()
        count_persons = 0
        if len(find_sensitive_words(row['email_text'])) > 0:
            result.i
        if row['quids'] and len(row['quids']) > 0: 
            return 'Sensitive'
        elif row['sensitive'] and len(row['sensitive']) > 0:
            return 'Sensitive'
        elif row['potentially_sensitive'] and len(row['potentially_sensitive']) > 0:
            for ent in row['potentially_sensitive']:
                if ent[1] == 'DATE':
                    continue
                elif ent[1] == 'TIME':
                    continue
                elif ent[1] == 'QUANTITY':
                    continue
                elif ent[1] == 'ORDINAL':
                    continue
                elif ent[1] == 'CARDINAL':
                    if len(find_all_phone_numbers(ent[0])) > 0 or len(find_all_credit_cards(ent[0])) > 0 or len(find_all_ssn(ent[0])) > 0:
                        return 'Sensitive'
                elif ent[1] == 'PERSON':
                    count_persons += 1
                    if count_persons > 2:
                        return 'Sensitive'
                    continue
            return 'Non-sensitive'
        else:
            return 'Non-sensitive'
    
    def analyze(self, row: pd.DataFrame) -> ContentAnalysisResult:
        result = ContentAnalysisResult()
        copied_row = row.copy()
        copied_row['entities'] = copied_row['email_text'].apply(self._get_entities)
        unique_entities = set()
    
        for entities in copied_row['entities']:
            for ent in entities:
                unique_entities.add(ent[1])
                
        copied_row['sensitive'] = copied_row['entities'].apply(lambda x: [(ent[0], ent[1]) for ent in x if ent[1] in ContentAnalyzer.SENSITIVE])
        copied_row['quids'] = copied_row['entities'].apply(lambda x: [(ent[0], ent[1]) for ent in x if ent[1] in ContentAnalyzer.QUIDS])
        copied_row['potentially_sensitive'] = copied_row['entities'].apply(lambda x: [(ent[0], ent[1]) for ent in x if ent[1] in ContentAnalyzer.POTENTIALLY_SENSITIVE])
        


In [ ]:
import pickle
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
import os

class TopicAnalyzer:
    # todo efi
    def __init__(self):
        self.model: LdaMulticore = LdaMulticore.load(os.path.join('lda', 'lda_model'))
        self.dictionary: Dictionary = Dictionary.load(os.path.join('lda', 'dictionary'))
        self.rule_to_topic =  {
            'None' : [38, 35, 39, 17, 33],
            '1.1' : [9, 35, 7, 20, 29], # legal
            '1.2': [1, 9, 7, 18, 20, 35, 24] + [5], # financial data
            '1.3': [1, 35, 9, 7, 39] + [5], # business or financial
            '2.1': [1, 9, 7, 35, 24, 20], # financial data
            '2.2': [1, 9, 18, 7, 17, 16, 24] # business data
        }
        
    def predict(self, row) -> TopicAnalysisResult:
        result = TopicAnalysisResult()
        result.is_legal = self.is_legal(row)
        result.is_business = self.is_business(row)
        result.is_finance = self.is_finance(row)
        
        return result

    def is_business(self, row):
        
        pass
    
    
    def is_finance(self, row):
        pass
    
    
    def is_legal(self, row):
        pass
    
    

In [1]:
def _only_1_rules_are_violated(static_rule_violations_by_rule_id):
    rule_2_prefix = '2.'
    
    for rule_id, rule_violation in static_rule_violations_by_rule_id:
        if rule_id.contains(rule_2_prefix) and static_rule_violations_by_rule_id[rule_id]:
            return False
        
    return True


def only_this_rule_violated(static_rule_violations_by_rule_id, rule_id):
    return static_rule_violations_by_rule_id[rule_id] == True and sum(static_rule_violations_by_rule_id.values())


class Enforcer:

    def __init__(self):
        self.static_analyzer = StaticAnalyzer()
        self.topic_analyzer = TopicAnalyzer()
        self.content_analyzer = ContentAnalyzer()
        
    def _pre_process(self, mail_row):
        # todo efi
        pass
    
        
    def enforce(self, mail_row) -> EnforcerResult:
        result = EnforcerResult()
        processed_row = self._pre_process(mail_row)
        
        static_rule_violations_by_rule_id: {str : bool} = self.analyze_static(processed_row)
        content_analysis: ContentAnalysisResult = None
        topic_analysis: TopicAnalysisResult = None
    
        # static analysis
        if any(static_rule_violations_by_rule_id.values()) is False:
            print(f'After static analysis, The Email {mail_row} is not violating any rule and therefore ALLOWED')
            return result
        
        if only_this_rule_violated(static_rule_violations_by_rule_id, rule_id='2,3'):
            print(f'Only rule 2.3 is potentially violated in {mail_row}, therefore, we can skip topic analysis')
            content_analysis: ContentAnalysisResult = self.content_analyzer.analyze(processed_row)
            
            if content_analysis.is_sensitive('2.3'):
                print(f'Email {mail_row} is violating rule 2.3 due to content analysis: {content_analysis} and therefore BLOCKED')
                result.violated_rules.append('2.3')
            else:
                print(f'Email {mail_row} is not violating any rule due to content analysis: {content_analysis} and therefore ALLOWED')
            return result
        
        print(f'Will analyze topic for {mail_row}, as all rules left required topic analysis')
        topic_analysis = self.topic_analyzer.predict(processed_row)
        
        if _only_1_rules_are_violated(static_rule_violations_by_rule_id):
            print(f'Only Policy 1# rules are violated statically, can skip content analysis')
        else:
            print(f'Will analyze content for {mail_row}, as all rules left required content analysis (2)')
            content_analysis: ContentAnalysisResult = self.content_analyzer.analyze(processed_row)
        
        for rule_id, static_rule_violation in static_rule_violations_by_rule_id.items():
            if static_rule_violation:
                print(f'Email {mail_row} is violating rule {rule_id} due to static analysis. Checking content analysis and topic analysis if needed')
            
                is_violated_rule_by_content_analysis = self._is_violating_rule_by_content(rule_id, content_analysis)
                is_violated_rule_by_topic_analysis = self._is_violating_rule_by_topic(rule_id, topic_analysis)
            
                if is_violated_rule_by_content_analysis and is_violated_rule_by_topic_analysis:
                    print(f'Email {mail_row} is violating rule {rule_id} due to content analysis or topic analysis and therefore BLOCKED')
                    result.violated_rules.append(rule_id)
                else:
                    print(f'Email {mail_row} is not violating rule {rule_id} due to content analysis or topic analysis and therefore ALLOWED')
        
        return result

    def analyze_static(self, processed_row) -> {str : bool}:
        static_rule_violations_by_rule_id: {str: bool} = {
            '1.1': self.static_analyzer.is_violating_rule_1_1(processed_row),
            '1.2': self.static_analyzer.is_violating_rule_1_2(processed_row),
            '1.3': self.static_analyzer.is_violating_rule_1_3(processed_row),
            '2.1': self.static_analyzer.is_violating_rule_2_1(processed_row),
            '2.2': self.static_analyzer.is_violating_rule_2_2(processed_row),
            '2.3': self.static_analyzer.is_violating_rule_2_3(processed_row)
        }
        
        return static_rule_violations_by_rule_id

    @staticmethod
    def _is_violating_rule_by_content(rule_id, content_analysis: ContentAnalysisResult):
        if rule_id.contains('1.'):
            return False
        if rule_id is '2.1' or rule_id is '2.2':
            return content_analysis.is_sensitive
        elif rule_id is '2.3':
            return content_analysis.is_sensitive(rule_id)
        else:
            raise Exception(f'Rule {rule_id} is not supported')
        
        
    @staticmethod
    def _is_violating_rule_by_topic(rule_id, topic_analysis: TopicAnalysisResult):
        if rule_id is '1.1':
            return topic_analysis.is_legal
        elif rule_id is '1.2':
            return topic_analysis.is_finance
        elif rule_id is '1.3':
            return topic_analysis.is_finance or topic_analysis.is_business
        elif rule_id is '2.1':
            return topic_analysis.is_finance
        elif rule_id is '2.2':
            return topic_analysis.is_business
        elif rule_id is '2.3':
            return False
        else: 
            raise Exception(f'Rule {rule_id} is not supported for topic analysis')
        


enforcer = Enforcer()

def classify_mail(mail_row) -> bool:
    enforcer_results = enforcer.enforce(mail_row)
    
    pass
    


IndentationError: expected an indented block after 'if' statement on line 41 (4084019371.py, line 44)